In [1]:
import pandas as pd
import requests
from scrapy.http import HtmlResponse

import warnings
warnings.filterwarnings('ignore')
import re

TypeError: attrs() got an unexpected keyword argument 'eq'

In [ ]:
url = 'http://www.joburgmarket.co.za/dailyprices.php'

In [ ]:
html = requests.get(url).content

In [ ]:
response = HtmlResponse(url=url, body=html)

In [5]:
response.css('div#right2 > h1 ::text').extract()

['Daily Prices']

In [6]:
response.xpath('//div[@id="right2"]/h1//text()').extract()

['Daily Prices']

In [7]:
response.xpath('//div[@id="right2"]').css('h1 ::text').extract_first()

'Daily Prices'

In [8]:
response.xpath('//div[@id="right2"]').css('p b ::text').extract_first()

'5 May 2020'

In [9]:
response.xpath('//div[@id="right2"]/h1//text()').get()

'Daily Prices'

In [10]:
response.xpath('//div[@id="right2"]').css('p b ::text').get()

'5 May 2020'

In [34]:
#response.css('select option ::text').getall()[1:]

In [40]:
sample = response.xpath('//select/option[@value]').extract()[1:5]

In [41]:
sample

['<option value="112">AMADUMBE</option>',
 '<option value="90">APPLES</option>',
 '<option value="113">ARTICHOKES</option>',
 '<option value="272">ASPARAGUS</option>']

In [47]:
for item in sample:
    print(re.findall(r'\d+', item))

['112']
['90']
['113']
['272']


In [ ]:
#link?commodity={}

In [99]:
col_headings = [
    'Date',
    'Commodity',
    'Total Value Sold',
    'Total Value Sold MTD',
    'Total Qty Sold',
    'Total Qty Sold MTD',
    'Total Kg Sold',
    'Total Kg Sold MTD',
    'Qty Available'
]

In [17]:
product = response.xpath('//table[@class="alltable"]').css('td.tleft2 ::text').extract()

In [18]:
product

['AMADUMBE',
 'APPLES',
 'ARTICHOKES',
 'ASPARAGUS',
 'ATCHARA',
 'AVOCADOS',
 'BABY BEET',
 'BABY BRINJALS',
 'BABY BUTTERNUT',
 'BABY CABBAGE',
 'BABY CARROTS',
 'BABY ENGLISH CUCUMBER',
 'BABY GEM SQUASH',
 'BABY LEAF',
 'BABY LEEKS',
 'BABY MARROWS',
 'BABY MIX',
 'BABY SWEET CORN',
 'BANANAS',
 'BASIL',
 'BEAN SPROUTS',
 'BEANS',
 'BEANS INDIAN',
 'BEETROOT',
 'BEETROOT PROCESSED',
 'BLACKBERRIES',
 'BLUEBERRIES',
 'BOK CHOY PAK-CHOI',
 'BRINJALS',
 'BROCCOLI',
 'BROCCOLI LEAVES',
 'BUTTER LETTUCE',
 'BUTTERNUT',
 'CABBAGE',
 'CABBAGE  SUGAR',
 'CABBAGE RED',
 'CALABASH',
 'CARROTS',
 'CAULI/BROC',
 'CAULIFLOWER',
 'CELERY',
 'CHILLIES',
 'CHINESE  VEGETABLES',
 'CHIPS',
 'CHIVES',
 'CHORIE',
 'CLEMENTINES',
 'COCKTAIL TOMATOES',
 'COCONUTS',
 'CORIANDER/DHANYA',
 'CRUSHED GINGER/GARLIC',
 'CUCUMBER',
 'CURRY LEAVES',
 'DATES',
 'DHANYA',
 'DILL',
 'DRAGON FRUIT',
 'DRIED FRUIT',
 'EDIBLE FLOWERS',
 'EMPTY BOXES',
 'ENDIVE/CHICORY',
 'ENGLISH CUCUMBER',
 'FENNEL/ANISE',
 'FIGS',
 

In [94]:
date = response.xpath('//div[@id="right2"]').css('p b ::text').extract_first()

In [100]:
purchases = []
for i, fresh_produce in enumerate(product):
    row = response.xpath('//table[@class="alltable"]').css('td.tleft ::text').extract()[(i * 7):(i * 7) + 7]
    modified_list = [] 
    for index, item in enumerate(row):
        item = item.replace(",", "").replace("R", "").replace("MTD:", "") # i.e. converted e.g. R 1,234.56 --> 1234.56
        item = item.lstrip()
        modified_list.append(float(item))
    purchases.append([date] + [fresh_produce] + modified_list)

In [101]:
purchases[0]

['30 April 2020', 'AMADUMBE', 1330.0, 96510.0, 7.0, 226.0, 140.0, 4520.0, 0.0]

In [106]:
df = pd.DataFrame(data=purchases, columns=col_headings)

In [107]:
df.head()

,Date,Commodity,Total Value Sold,Total Value Sold MTD,Total Qty Sold,Total Qty Sold MTD,Total Kg Sold,Total Kg Sold MTD,Qty Available
0,30 April 2020,AMADUMBE,1330.00,96510.0,7.0,226.0,140.0,4520.0,0.0
1,30 April 2020,APPLES,1866501.00,28600166.3,23769.0,333449.0,303208.0,3969349.0,118915.0
2,30 April 2020,ASPARAGUS,32000.00,135806.0,32.0,382.0,160.0,1693.0,1.0
3,30 April 2020,ATCHARA,820.80,4551.8,29.0,122.0,38.0,215.0,196.0
4,30 April 2020,AVOCADOS,643910.82,10796817.8,7469.0,150080.0,61098.0,1002509.0,17553.0
